<a href="https://colab.research.google.com/github/farookfazni/AI-to-Predict-Career-Path/blob/master/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import pandas as pd
import numpy as np
import nltk

In [26]:
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras import metrics, regularizers
from keras.preprocessing import sequence

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [27]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
Data = pd.read_csv('/content/drive/MyDrive/Research Dataset/processed_data/Added_Target/FullyMerged.csv')

In [29]:
Data.head()

,Unnamed: 0,id,skills,edu_1,edu_2,edu_3,org_1,org_2,org_3,org_4,Target
0,0,gayanchinthaka,Java Android Development PHP HTML C Cascading ...,Computer Science and Technology Uva Wellassa U...,Diploma in English ESOFT Metro Campus,Diploma in Technology ESOFT Metro Campus,Software Engineer Typefi Systems,Associate Software Engineer Rezgateway,Internship Trainee Rezgateway,,Software Engineer
1,1,kashita-niranjan-udayanga-04106a47,...,BSc Engineering(Hons) University of Moratuwa,,,Software Engineer Atrenta,Undergraduate University of Moratuwa,Trainee Engineer Lanka Bell,,Software Engineer
2,2,lingeswaran-sivapiriyan-b90685177,JavaScript Reactjs MySQL ObjectOriented Progra...,B.Eng (Hons) Software Engineering London Metr...,,,Software Engineer Invicta Innovations,Trainee Samuel Gnanam IT Centre,,,Software Engineer
3,3,viraj-chamara-5b6751149,Java JavaScript PHP HTML Cascading Style Sheet...,Institute of Java and Software Engineering,,,Software Engineer I O N Groups Pvt Ltd,Software Engineer ites software pvt ltd,,,Software Engineer
4,4,iyani-kalupahana-5b781951,Java Android Development C Communication HTML ...,Honours Degree of Bachelor of the Science of E...,"High School Devi Balika Vidyalaya,Colombo 08",,Software Engineer Creative Software,Student University of Moratuwa,Researcher National University of Singapore,Intern University of Moratuwa,Software Engineer


In [30]:
Data = Data.drop(['Unnamed: 0'], axis=1)

In [31]:
Data.head()

,id,skills,edu_1,edu_2,edu_3,org_1,org_2,org_3,org_4,Target
0,gayanchinthaka,Java Android Development PHP HTML C Cascading ...,Computer Science and Technology Uva Wellassa U...,Diploma in English ESOFT Metro Campus,Diploma in Technology ESOFT Metro Campus,Software Engineer Typefi Systems,Associate Software Engineer Rezgateway,Internship Trainee Rezgateway,,Software Engineer
1,kashita-niranjan-udayanga-04106a47,...,BSc Engineering(Hons) University of Moratuwa,,,Software Engineer Atrenta,Undergraduate University of Moratuwa,Trainee Engineer Lanka Bell,,Software Engineer
2,lingeswaran-sivapiriyan-b90685177,JavaScript Reactjs MySQL ObjectOriented Progra...,B.Eng (Hons) Software Engineering London Metr...,,,Software Engineer Invicta Innovations,Trainee Samuel Gnanam IT Centre,,,Software Engineer
3,viraj-chamara-5b6751149,Java JavaScript PHP HTML Cascading Style Sheet...,Institute of Java and Software Engineering,,,Software Engineer I O N Groups Pvt Ltd,Software Engineer ites software pvt ltd,,,Software Engineer
4,iyani-kalupahana-5b781951,Java Android Development C Communication HTML ...,Honours Degree of Bachelor of the Science of E...,"High School Devi Balika Vidyalaya,Colombo 08",,Software Engineer Creative Software,Student University of Moratuwa,Researcher National University of Singapore,Intern University of Moratuwa,Software Engineer


In [32]:
Data['ALL'] = Data[['skills','edu_1','edu_2','edu_3','org_1','org_2','org_3','org_4']].agg(' '.join, axis=1)

In [33]:
Data

,id,skills,edu_1,edu_2,edu_3,org_1,org_2,org_3,org_4,Target,ALL
0,gayanchinthaka,Java Android Development PHP HTML C Cascading ...,Computer Science and Technology Uva Wellassa U...,Diploma in English ESOFT Metro Campus,Diploma in Technology ESOFT Metro Campus,Software Engineer Typefi Systems,Associate Software Engineer Rezgateway,Internship Trainee Rezgateway,,Software Engineer,Java Android Development PHP HTML C Cascading ...
1,kashita-niranjan-udayanga-04106a47,...,BSc Engineering(Hons) University of Moratuwa,,,Software Engineer Atrenta,Undergraduate University of Moratuwa,Trainee Engineer Lanka Bell,,Software Engineer,...
2,lingeswaran-sivapiriyan-b90685177,JavaScript Reactjs MySQL ObjectOriented Progra...,B.Eng (Hons) Software Engineering London Metr...,,,Software Engineer Invicta Innovations,Trainee Samuel Gnanam IT Centre,,,Software Engineer,JavaScript Reactjs MySQL ObjectOriented Progra...
3,viraj-chamara-5b6751149,Java JavaScript PHP HTML Cascading Style Sheet...,Institute of Java and Software Engineering,,,Software Engineer I O N Groups Pvt Ltd,Software Engineer ites software pvt ltd,,,Software Engineer,Java JavaScript PHP HTML Cascading Style Sheet...
4,iyani-kalupahana-5b781951,Java Android Development C Communication HTML ...,Honours Degree of Bachelor of the Science of E...,"High School Devi Balika Vidyalaya,Colombo 08",,Software Engineer Creative Software,Student University of Moratuwa,Researcher National University of Singapore,Intern University of Moratuwa,Software Engineer,Java Android Development C Communication HTML ...
...,...,...,...,...,...,...,...,...,...,...,...
4571,ematinov,SQL Git Machine Learning C Deep Learning Pytho...,"Master's degree, Applied Mathematics and Compu...","Bachelor's degree, Mathematics and Computer Sc...",,Deep Learning Engineer iTechArt Group,Data Scientist EPAM Systems,Deep Learning Engineer Oxagile,,Data Science,SQL Git Machine Learning C Deep Learning Pytho...
4572,waqqas-ansari-36395686,Python Programming Language Bootstrap Machine ...,"Bachelor of Technology (B.Tech), Computer Scie...","junior college, Science (Physics, chemistry, M...","High School, Science (Physics, chemistry, Math...",Software Trainee PYAPP TECHNOLOGIES PRIVATE LI...,Data Scientist The Sparks Foundation,,,Data Science,Python Programming Language Bootstrap Machine ...
4573,prabod-rathnayaka,Java C JavaScript PHP MySQL Cascading Style Sh...,"Doctor of Philosophy - PhD, Artificial Intelli...","Bachelor of Science (BSc) in Engineering, Comp...",,Teaching Assistant La Trobe University,Graduate Research Assistant La Trobe University,Data Scientist John Snow Labs,Research Engineer Singapore Management University,Data Science,Java C JavaScript PHP MySQL Cascading Style Sh...
4574,gnana-prasath,Python Programming Language HTML Machine Learn...,"Bachelor of Engineering (B.E.), Mechatronics, ...",,,Associate Data scientist TheMathCompany,Application development Senior Analyst Accentu...,Application Development Analyst Accenture in I...,Associate Software Engineer Accenture in India,Data Science,Python Programming Language HTML Machine Learn...


In [34]:
def check_Missing(dataset, precent):
  #Checking columns with missing data
  Miss_Percent=100*(dataset.isnull().sum()/len(dataset))

  #Sorting the data columns by their percentage in descending order (big to small)
  # by using Miss_Percent>0 it takes values,ore than 0
  # round(1) this is used to round the value to one decimal point .0
  Miss_Percent=Miss_Percent[Miss_Percent>precent].sort_values(ascending=False).round(1)

  #Creating a dataframe to show percentage of missing data and its respective data column in table
  DataFrame=pd.DataFrame(Miss_Percent)
  miss_percent_table=DataFrame.rename(columns={0:'% of Missing Values'})
  MissPercent=miss_percent_table

  #Displaying Missing Value table
  return MissPercent

In [35]:
Data_New = Data.replace(r'^\s*$', np.nan, regex=True)

In [36]:
MissPercent = check_Missing(Data_New,0)

In [37]:
MissPercent

,% of Missing Values
edu_3,62.8
org_4,47.7
edu_2,30.3
org_3,26.3
org_2,8.9
edu_1,7.3
skills,4.3


In [45]:
Data_all = Data.iloc[:,9:]
Data_all

,Target,ALL
0,Software Engineer,Java Android Development PHP HTML C Cascading ...
1,Software Engineer,...
2,Software Engineer,JavaScript Reactjs MySQL ObjectOriented Progra...
3,Software Engineer,Java JavaScript PHP HTML Cascading Style Sheet...
4,Software Engineer,Java Android Development C Communication HTML ...
...,...,...
4571,Data Science,SQL Git Machine Learning C Deep Learning Pytho...
4572,Data Science,Python Programming Language Bootstrap Machine ...
4573,Data Science,Java C JavaScript PHP MySQL Cascading Style Sh...
4574,Data Science,Python Programming Language HTML Machine Learn...


In [46]:
Data_all['ALL'][1]

'                                                               BSc Engineering(Hons) University of Moratuwa     Software Engineer Atrenta Undergraduate University of Moratuwa Trainee Engineer Lanka Bell  '

In [47]:
Data_all['tokenized_ALL'] = Data_all.apply(lambda row: nltk.word_tokenize(row['ALL']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
Data_all

,Target,ALL,tokenized_ALL
0,Software Engineer,Java Android Development PHP HTML C Cascading ...,"[Java, Android, Development, PHP, HTML, C, Cas..."
1,Software Engineer,...,"[BSc, Engineering, (, Hons, ), University, of,..."
2,Software Engineer,JavaScript Reactjs MySQL ObjectOriented Progra...,"[JavaScript, Reactjs, MySQL, ObjectOriented, P..."
3,Software Engineer,Java JavaScript PHP HTML Cascading Style Sheet...,"[Java, JavaScript, PHP, HTML, Cascading, Style..."
4,Software Engineer,Java Android Development C Communication HTML ...,"[Java, Android, Development, C, Communication,..."
...,...,...,...
4571,Data Science,SQL Git Machine Learning C Deep Learning Pytho...,"[SQL, Git, Machine, Learning, C, Deep, Learnin..."
4572,Data Science,Python Programming Language Bootstrap Machine ...,"[Python, Programming, Language, Bootstrap, Mac..."
4573,Data Science,Java C JavaScript PHP MySQL Cascading Style Sh...,"[Java, C, JavaScript, PHP, MySQL, Cascading, S..."
4574,Data Science,Python Programming Language HTML Machine Learn...,"[Python, Programming, Language, HTML, Machine,..."


In [49]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(Data_all['Target'])
# Data_tokenized.drop("Target", axis=1, inplace=True)
Data_all["Target_NO"] = label
Data_all

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Target,ALL,tokenized_ALL,Target_NO
0,Software Engineer,Java Android Development PHP HTML C Cascading ...,"[Java, Android, Development, PHP, HTML, C, Cas...",9
1,Software Engineer,...,"[BSc, Engineering, (, Hons, ), University, of,...",9
2,Software Engineer,JavaScript Reactjs MySQL ObjectOriented Progra...,"[JavaScript, Reactjs, MySQL, ObjectOriented, P...",9
3,Software Engineer,Java JavaScript PHP HTML Cascading Style Sheet...,"[Java, JavaScript, PHP, HTML, Cascading, Style...",9
4,Software Engineer,Java Android Development C Communication HTML ...,"[Java, Android, Development, C, Communication,...",9
...,...,...,...,...
4571,Data Science,SQL Git Machine Learning C Deep Learning Pytho...,"[SQL, Git, Machine, Learning, C, Deep, Learnin...",3
4572,Data Science,Python Programming Language Bootstrap Machine ...,"[Python, Programming, Language, Bootstrap, Mac...",3
4573,Data Science,Java C JavaScript PHP MySQL Cascading Style Sh...,"[Java, C, JavaScript, PHP, MySQL, Cascading, S...",3
4574,Data Science,Python Programming Language HTML Machine Learn...,"[Python, Programming, Language, HTML, Machine,...",3


In [13]:
Data['tokenized_skills'] = Data.apply(lambda row: nltk.word_tokenize(row['skills']), axis=1)

In [14]:
Data['tokenized_edu1'] = Data.apply(lambda row: nltk.word_tokenize(row['edu_1']), axis=1)
Data['tokenized_edu2'] = Data.apply(lambda row: nltk.word_tokenize(row['edu_2']), axis=1)
Data['tokenized_edu3'] = Data.apply(lambda row: nltk.word_tokenize(row['edu_3']), axis=1)
Data['tokenized_org1'] = Data.apply(lambda row: nltk.word_tokenize(row['org_1']), axis=1)
Data['tokenized_org2'] = Data.apply(lambda row: nltk.word_tokenize(row['org_2']), axis=1)
Data['tokenized_org3'] = Data.apply(lambda row: nltk.word_tokenize(row['org_3']), axis=1)

In [15]:
Data['tokenized_Target'] = Data.apply(lambda row: nltk.word_tokenize(row['Target']), axis=1)

In [16]:
Data

,id,skills,edu_1,edu_2,edu_3,org_1,org_2,org_3,org_4,Target,tokenized_skills,tokenized_edu1,tokenized_edu2,tokenized_edu3,tokenized_org1,tokenized_org2,tokenized_org3,tokenized_Target
0,gayanchinthaka,Java Android Development PHP HTML C Cascading ...,Computer Science and Technology Uva Wellassa U...,Diploma in English ESOFT Metro Campus,Diploma in Technology ESOFT Metro Campus,Software Engineer Typefi Systems,Associate Software Engineer Rezgateway,Internship Trainee Rezgateway,,Software Engineer,"[Java, Android, Development, PHP, HTML, C, Cas...","[Computer, Science, and, Technology, Uva, Well...","[Diploma, in, English, ESOFT, Metro, Campus]","[Diploma, in, Technology, ESOFT, Metro, Campus]","[Software, Engineer, Typefi, Systems]","[Associate, Software, Engineer, Rezgateway]","[Internship, Trainee, Rezgateway]","[Software, Engineer]"
1,kashita-niranjan-udayanga-04106a47,...,BSc Engineering(Hons) University of Moratuwa,,,Software Engineer Atrenta,Undergraduate University of Moratuwa,Trainee Engineer Lanka Bell,,Software Engineer,[],"[BSc, Engineering, (, Hons, ), University, of,...",[],[],"[Software, Engineer, Atrenta]","[Undergraduate, University, of, Moratuwa]","[Trainee, Engineer, Lanka, Bell]","[Software, Engineer]"
2,lingeswaran-sivapiriyan-b90685177,JavaScript Reactjs MySQL ObjectOriented Progra...,B.Eng (Hons) Software Engineering London Metr...,,,Software Engineer Invicta Innovations,Trainee Samuel Gnanam IT Centre,,,Software Engineer,"[JavaScript, Reactjs, MySQL, ObjectOriented, P...","[B.Eng, (, Hons, ), Software, Engineering, Lon...",[],[],"[Software, Engineer, Invicta, Innovations]","[Trainee, Samuel, Gnanam, IT, Centre]",[],"[Software, Engineer]"
3,viraj-chamara-5b6751149,Java JavaScript PHP HTML Cascading Style Sheet...,Institute of Java and Software Engineering,,,Software Engineer I O N Groups Pvt Ltd,Software Engineer ites software pvt ltd,,,Software Engineer,"[Java, JavaScript, PHP, HTML, Cascading, Style...","[Institute, of, Java, and, Software, Engineering]",[],[],"[Software, Engineer, I, O, N, Groups, Pvt, Ltd]","[Software, Engineer, ites, software, pvt, ltd]",[],"[Software, Engineer]"
4,iyani-kalupahana-5b781951,Java Android Development C Communication HTML ...,Honours Degree of Bachelor of the Science of E...,"High School Devi Balika Vidyalaya,Colombo 08",,Software Engineer Creative Software,Student University of Moratuwa,Researcher National University of Singapore,Intern University of Moratuwa,Software Engineer,"[Java, Android, Development, C, Communication,...","[Honours, Degree, of, Bachelor, of, the, Scien...","[High, School, Devi, Balika, Vidyalaya, ,, Col...",[],"[Software, Engineer, Creative, Software]","[Student, University, of, Moratuwa]","[Researcher, National, University, of, Singapore]","[Software, Engineer]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,ematinov,SQL Git Machine Learning C Deep Learning Pytho...,"Master's degree, Applied Mathematics and Compu...","Bachelor's degree, Mathematics and Computer Sc...",,Deep Learning Engineer iTechArt Group,Data Scientist EPAM Systems,Deep Learning Engineer Oxagile,,Data Science,"[SQL, Git, Machine, Learning, C, Deep, Learnin...","[Master, 's, degree, ,, Applied, Mathematics, ...","[Bachelor, 's, degree, ,, Mathematics, and, Co...",[],"[Deep, Learning, Engineer, iTechArt, Group]","[Data, Scientist, EPAM, Systems]","[Deep, Learning, Engineer, Oxagile]","[Data, Science]"
4572,waqqas-ansari-36395686,Python Programming Language Bootstrap Machine ...,"Bachelor of Technology (B.Tech), Computer Scie...","junior college, Science (Physics, chemistry, M...","High School, Science (Physics, chemistry, Math...",Software Trainee PYAPP TECHNOLOGIES PRIVATE LI...,Data Scientist The Sparks Foundation,,,Data Science,"[Python, Programming, Language, Bootstrap, Mac...","[Bachelor, of, Technology, (, B.Tech, ), ,, Co...","[junior, college, ,, Science, (, Physics, ,, c...","[High, School, ,, Science, (, Physics, ,, chem...","[Software,

In [17]:
Data_tokenized = Data.iloc[:,9:]

In [18]:
Data_tokenized

,Target,tokenized_skills,tokenized_edu1,tokenized_edu2,tokenized_edu3,tokenized_org1,tokenized_org2,tokenized_org3,tokenized_Target
0,Software Engineer,"[Java, Android, Development, PHP, HTML, C, Cas...","[Computer, Science, and, Technology, Uva, Well...","[Diploma, in, English, ESOFT, Metro, Campus]","[Diploma, in, Technology, ESOFT, Metro, Campus]","[Software, Engineer, Typefi, Systems]","[Associate, Software, Engineer, Rezgateway]","[Internship, Trainee, Rezgateway]","[Software, Engineer]"
1,Software Engineer,[],"[BSc, Engineering, (, Hons, ), University, of,...",[],[],"[Software, Engineer, Atrenta]","[Undergraduate, University, of, Moratuwa]","[Trainee, Engineer, Lanka, Bell]","[Software, Engineer]"
2,Software Engineer,"[JavaScript, Reactjs, MySQL, ObjectOriented, P...","[B.Eng, (, Hons, ), Software, Engineering, Lon...",[],[],"[Software, Engineer, Invicta, Innovations]","[Trainee, Samuel, Gnanam, IT, Centre]",[],"[Software, Engineer]"
3,Software Engineer,"[Java, JavaScript, PHP, HTML, Cascading, Style...","[Institute, of, Java, and, Software, Engineering]",[],[],"[Software, Engineer, I, O, N, Groups, Pvt, Ltd]","[Software, Engineer, ites, software, pvt, ltd]",[],"[Software, Engineer]"
4,Software Engineer,"[Java, Android, Development, C, Communication,...","[Honours, Degree, of, Bachelor, of, the, Scien...","[High, School, Devi, Balika, Vidyalaya, ,, Col...",[],"[Software, Engineer, Creative, Software]","[Student, University, of, Moratuwa]","[Researcher, National, University, of, Singapore]","[Software, Engineer]"
...,...,...,...,...,...,...,...,...,...
4571,Data Science,"[SQL, Git, Machine, Learning, C, Deep, Learnin...","[Master, 's, degree, ,, Applied, Mathematics, ...","[Bachelor, 's, degree, ,, Mathematics, and, Co...",[],"[Deep, Learning, Engineer, iTechArt, Group]","[Data, Scientist, EPAM, Systems]","[Deep, Learning, Engineer, Oxagile]","[Data, Science]"
4572,Data Science,"[Python, Programming, Language, Bootstrap, Mac...","[Bachelor, of, Technology, (, B.Tech, ), ,, Co...","[junior, college, ,, Science, (, Physics, ,, c...","[High, School, ,, Science, (, Physics, ,, chem...","[Software, Trainee, PYAPP, TECHNOLOGIES, PRIVA...","[Data, Scientist, The, Sparks, Foundation]",[],"[Data, Science]"
4573,Data Science,"[Java, C, JavaScript, PHP, MySQL, Cascading, S...","[Doctor, of, Philosophy, -, PhD, ,, Artificial...","[Bachelor, of, Science, (, BSc, ), in, Enginee...",[],"[Teaching, Assistant, La, Trobe, University]","[Graduate, Research, Assistant, La, Trobe, Uni...","[Data, Scientist, John, Snow, Labs]","[Data, Science]"
4574,Data Science,"[Python, Programming, Language, HTML, Machine,...","[Bachelor, of, Engineering, (, B.E, ., ), ,, M...",[],[],"[Associate, Data, scientist, TheMathCompany]","[Application, development, Senior, Analyst, Ac...","[Application, Development, Analyst, Accenture,...","[Data, Science]"


In [19]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# from nltk.stem import wordnet
# from nltk.stem import WordNetLemmatizer
# word_lem = WordNetLemmatizer()

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(Data_tokenized['Target'])
label

array([9, 9, 9, ..., 3, 3, 3])

In [21]:
Data_tokenized.drop("Target", axis=1, inplace=True)
Data_tokenized["Target"] = label
Data_tokenized

,tokenized_skills,tokenized_edu1,tokenized_edu2,tokenized_edu3,tokenized_org1,tokenized_org2,tokenized_org3,tokenized_Target,Target
0,"[Java, Android, Development, PHP, HTML, C, Cas...","[Computer, Science, and, Technology, Uva, Well...","[Diploma, in, English, ESOFT, Metro, Campus]","[Diploma, in, Technology, ESOFT, Metro, Campus]","[Software, Engineer, Typefi, Systems]","[Associate, Software, Engineer, Rezgateway]","[Internship, Trainee, Rezgateway]","[Software, Engineer]",9
1,[],"[BSc, Engineering, (, Hons, ), University, of,...",[],[],"[Software, Engineer, Atrenta]","[Undergraduate, University, of, Moratuwa]","[Trainee, Engineer, Lanka, Bell]","[Software, Engineer]",9
2,"[JavaScript, Reactjs, MySQL, ObjectOriented, P...","[B.Eng, (, Hons, ), Software, Engineering, Lon...",[],[],"[Software, Engineer, Invicta, Innovations]","[Trainee, Samuel, Gnanam, IT, Centre]",[],"[Software, Engineer]",9
3,"[Java, JavaScript, PHP, HTML, Cascading, Style...","[Institute, of, Java, and, Software, Engineering]",[],[],"[Software, Engineer, I, O, N, Groups, Pvt, Ltd]","[Software, Engineer, ites, software, pvt, ltd]",[],"[Software, Engineer]",9
4,"[Java, Android, Development, C, Communication,...","[Honours, Degree, of, Bachelor, of, the, Scien...","[High, School, Devi, Balika, Vidyalaya, ,, Col...",[],"[Software, Engineer, Creative, Software]","[Student, University, of, Moratuwa]","[Researcher, National, University, of, Singapore]","[Software, Engineer]",9
...,...,...,...,...,...,...,...,...,...
4571,"[SQL, Git, Machine, Learning, C, Deep, Learnin...","[Master, 's, degree, ,, Applied, Mathematics, ...","[Bachelor, 's, degree, ,, Mathematics, and, Co...",[],"[Deep, Learning, Engineer, iTechArt, Group]","[Data, Scientist, EPAM, Systems]","[Deep, Learning, Engineer, Oxagile]","[Data, Science]",3
4572,"[Python, Programming, Language, Bootstrap, Mac...","[Bachelor, of, Technology, (, B.Tech, ), ,, Co...","[junior, college, ,, Science, (, Physics, ,, c...","[High, School, ,, Science, (, Physics, ,, chem...","[Software, Trainee, PYAPP, TECHNOLOGIES, PRIVA...","[Data, Scientist, The, Sparks, Foundation]",[],"[Data, Science]",3
4573,"[Java, C, JavaScript, PHP, MySQL, Cascading, S...","[Doctor, of, Philosophy, -, PhD, ,, Artificial...","[Bachelor, of, Science, (, BSc, ), in, Enginee...",[],"[Teaching, Assistant, La, Trobe, University]","[Graduate, Research, Assistant, La, Trobe, Uni...","[Data, Scientist, John, Snow, Labs]","[Data, Science]",3
4574,"[Python, Programming, Language, HTML, Machine,...","[Bachelor, of, Engineering, (, B.E, ., ), ,, M...",[],[],"[Associate, Data, scientist, TheMathCompany]","[Application, development, Senior, Analyst, Ac...","[Application, Development, Analyst, Accenture,...","[Data, Science]",3


In [ ]:
#Split the dataset to Training and Test subsets (90/10)
train, test = train_test_split(Data_tokenized, test_size = 0.1, random_state = 17) #random_state = None

In [ ]:
train_descs = train.iloc[:,:7]
train_labels = train['Target']
 
test_descs = test.iloc[:,:7]
test_labels = test['Target']

In [ ]:
train_descs

,tokenized_skills,tokenized_edu1,tokenized_edu2,tokenized_edu3,tokenized_org1,tokenized_org2,tokenized_org3,tokenized_Target
2989,"[JavaScript, PHP, HTML, Software, Development,...","[HND, in, Computing, and, System, Development,...","[Cisco, Certified, Network, Associate, LAVTC, ...",[],"[DevOps, Engineer, Exetel]","[Flutter, Mobile, Frontend, Developer, The, Ri...","[DevOps, Engineer, PickMe, (, Digital, Mobilit...",[DevOps]
1734,"[Python, Programming, Language, PHP, MySQL, HT...","[BSC, (, Hons, ), in, Information, Technology,...","[Basic, ’, s, education, ,, O/L, ,, A/L, Royal...",[],"[Graphic, Designer, SLIIT, Faculty, of, Comput...",[],[],"[Cyber, Security]"
3590,"[C, Linux, Machine, Learning, Programming, C, ...","[Master, of, Technology, by, Research, ,, Comp...","[Bachelor, ’, s, Degree, ,, Computer, Engineer...",[],"[AI/ML, Engineer, Verisk]","[Deep, Learning, Nanodegree, Project, Reviewer...","[Research, Intern, Intel, Labs]","[Machine, Learning, Engineer]"
606,"[Software, Quality, Assurance, API, Testing, A...","[Undergraduate, University, of, Colombo]","[Sandalanka, National, College]",[],"[Quality, Assurance, Engineer, 1, Billion, Tech]","[Private, Tutor, Self, Employed]","[Testing, Associate, RR, Donnelley]","[Quality, Assurance]"
3333,"[Java, C, JavaScript, PHP, MySQL, Cascading, S...","[SparkX, Proffessional, Development, Program-2...","[Sir, John, Kothalawala, M.V, .]","[Ku/Udabadalawa, Sudarshana, M.V, .]","[Software, Engineer, Sysco, LABS, Sri, Lanka]","[Computer, Engineer, Intern, Dialog, Axiata, PLC]","[Junior, Treasurer, SEDS, Pera]","[Software, Engineer]"
...,...,...,...,...,...,...,...,...
2800,"[Java, JavaScript, PHP, C, Cascading, Style, S...","[Master, of, Science, -, MS, Binghamton, Unive...","[Bachelor, of, Engineering, (, B.E, ., ), Vish...",[],"[Software, Developer, Johnson, &, Johnson]","[Full, Stack, Developer, Global, Health, Impac...","[Student, Employee, Sodexo]","[Mobile, App, Developer]"
1337,"[Leadership, Project, Management, Microsoft, O...","[Bachelor, of, Science, (, Honors, ), Universi...","[Specialization, University, of, Illinois, at,...","[Lyceum, International, School]","[Junior, Business, Analyst, 3rive, Technologies]","[Global, Corporate, Projects, Brandix]","[Customer, Service, Representative, Domino, 's...","[Business, Analyst]"
406,"[Java, JavaScript, Android, Development, PHP, ...","[Diploma, in, Information, Technology, SLIIT]","[Galahitiyawa, Central, College]","[Grade, 5, scholarship, Bandarawaththa, Parakr...","[Associate, Software, Engineer, Sampath, Infor...","[Software, Engineer, Intern, Sampath, Informat...","[Freelance, Backend, Developer, Self-employed]","[Software, Engineer]"
2191,"[Java, C, Programming, Python, Programming, La...","[Master, of, Business, Administration, -, MBA,...","[Bachelor, of, Science, (, BSc, ), University,...",[],"[Senior, Network, Engineer, -, Global, Network...","[Assistant, Manager, -, Voice, Core, Operation...","[Senior, Engineer, -, Voice, Core, Operations,...","[Network, Engineer]"


In [ ]:
test_labels

1401    0
4510    3
744     8
3656    5
2684    6
       ..
1986    7
3396    9
3964    2
3905    2
4054    2
Name: Target, Length: 458, dtype: int64

In [ ]:
train_descs.shape

(4118, 7)

In [ ]:
test_descs.shape

(458, 7)

In [ ]:
# Model Parameters
vocab_size = 1000

sequences_length = 1200

embedding_dimensionality = 64 #possibly low??
max_features = 2000 #equal to vocab_size

# num_labels = len(train_labels.unique())
batch_size = 32
nb_epoch = 20

nof_filters = 200
kernel_size = 16

hidden_dims = 512

In [ ]:
# # Convert Texts to Numeric Vectors for Input
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(train_descs)

# x_train = tokenizer.texts_to_sequences(train_descs)
# x_test = tokenizer.texts_to_sequences(test_descs)

# x_train = sequence.pad_sequences(x_train, padding = 'post')
# x_test = sequence.pad_sequences(x_test, padding = 'post')

# encoder = LabelBinarizer()
# encoder.fit(train_labels)
# y_train = encoder.transform(train_labels)
# y_test = encoder.transform(test_labels)

In [ ]:
# tokenizer

In [ ]:
# x_train

In [ ]:
model = Sequential()

model.add(Embedding(4118,128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))

# model.add(Conv1D(nof_filters, kernel_size, padding='valid', activation='relu', strides = 1))
# model.add(GlobalMaxPooling1D())

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(32))
model.add(Dropout(0.2))

model.add(Dense(1,activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', #'sgd', 'adam', 'RMSprop', 'Adagrad'
                   metrics = [metrics.categorical_accuracy])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         527104    
                                                                 
 lstm_3 (LSTM)               (None, None, 128)         131584    
                                                                 
 dropout_3 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_4 (LSTM)               (None, None, 128)         131584    
                                                                 
 dropout_4 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_5 (LSTM)               (None, 32)                20608     
                                                                 
 dropout_5 (Dropout)         (None, 32)               

In [ ]:
history = model.fit(train_descs, train_labels,
                    batch_size = batch_size,
                    epochs = nb_epoch,
                    verbose = True,
                    validation_split = 0.2)

ValueError: ignored

In [ ]:
score = model.evaluate(x_test, y_test, batch_size = batch_size, verbose = True)
 
print('\nTest categorical_crossentropy:', score[0])
print('Categorical accuracy:', score[1])

ValueError: ignored